# 2. CNN으로 MNIST 분류하기

## 1) 모델 이해하기

### 1-1. 1번째 표기방법

- 합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명

### 1-2. 2번째 표기 방법

- 합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 보는 방법

## 2) 모델 구현하기

### 2-1. 라이브러리와 입력값 정의

In [1]:
import torch
import torch.nn as nn

- 임의의 텐서 만들기
- 텐서 크기는 1 x 1 x 28 x 28(배치 크기 x 채널 x 높이 x 너비)

In [3]:
inputs = torch.Tensor(1,1,28,28)
print(f'텐서의 크기 : {inputs.shape}')

텐서의 크기 : torch.Size([1, 1, 28, 28])


### 2-2. 합성곱층과 풀링 선언하기

In [4]:
conv1 = nn.Conv2d(1,32,3,padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


- 채널 1개짜리를 입력 받아 32채널을 뽑아내는데 커널 사이즈는 3이고 padding은 1이다.

In [6]:
conv2 = nn.Conv2d(32,64,kernel_size = 3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


- 32채널 입력 받아 64개의 채널 뽑고 커널 사이즈 3이고 Padding은 1이다.

In [7]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


- 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정

### 2-3. 구현체를 연결하여 모델 만들기

In [8]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [11]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [12]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [13]:
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [14]:
fc = nn.Linear(3136,10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## 3) CNN으로 MNIST 분류하기

In [15]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [16]:
torch.manual_seed(777)

In [17]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [19]:
train = dsets.MNIST(root='MNIST_data/',
                   train = True,
                   transform = transforms.ToTensor(),
                   download = True)

test = dsets.MNIST(root='MNIST_data/',
                   train = False,
                   transform = transforms.ToTensor(),
                   download = True)

In [20]:
data_loader = torch.utils.data.DataLoader(dataset = train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [23]:
class CNN(torch.nn.Module) :
    
    def __init__(self) :
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(
                torch.nn.Conv2d(1,32,kernel_size = 3, stride = 1, padding = 1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = torch.nn.Sequential(
                torch.nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
       
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
        
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x) :
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1) # 전결합층을 위해서 flatten
        out = self.fc(out)
        return out

In [24]:
model = CNN()

In [25]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [29]:
total_batch = len(data_loader)

print(f'총 배치의 수 : {total_batch}')

총 배치의 수 : 600


In [31]:
for epoch in range(training_epochs) :
    avg_cost = 0
    
    for X,y in data_loader :
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print(f'Epoch : {epoch+1} cost : {avg_cost}')

Epoch : 1 cost : 0.22022147476673126
Epoch : 2 cost : 0.05971866101026535
Epoch : 3 cost : 0.04462616890668869
Epoch : 4 cost : 0.036059796810150146
Epoch : 5 cost : 0.030205298215150833
Epoch : 6 cost : 0.024741632863879204
Epoch : 7 cost : 0.021128306165337563
Epoch : 8 cost : 0.018007254227995872
Epoch : 9 cost : 0.014921202324330807
Epoch : 10 cost : 0.012298358604311943
Epoch : 11 cost : 0.01123000867664814
Epoch : 12 cost : 0.008639082312583923
Epoch : 13 cost : 0.006994095165282488
Epoch : 14 cost : 0.007855444215238094
Epoch : 15 cost : 0.006686242297291756


### 테스트

In [32]:
with torch.no_grad() : # 학습 진행 X
    X_test = test.test_data.view(len(test),1,28,28).float()
    y_test = test.test_labels
    
    pred = model(X_test)
    correct_pred = torch.argmax(pred,1) == y_test
    acc = correct_pred.float().mean()
    print(f"Accuracy : {acc.item()}")

/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy : 0.986299991607666


# 3. 깊은 CNN으로 MNIST 분류하기

In [ ]:
class Deep_CNN(tormodulesnn.Module):
    def __init__(self) :
        super(Deep_CNN, self).__init__()
        self.keep_prob = 0.5 ## dropout 같음
        
        self.layer1 = torch.nn.Sequential(
                torch.nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding = 1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = torch.nn.Sequential(
                torch.nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer3 = torch.nn.Sequential(
                torch.nn.Conv2d(64,128, kernel_size = 3, stride = 1, padding = 1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
        
        self.fc1 = torch.nn.Linear(4*4*128, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
                self.fc1,
                torch.nn.ReLU(),
                torch.nn.Dropout(p = 1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625,10, bias = True)
        torch.nn.xavier_uniform_(self.fc2.weight)
        
        
    def forward(self, x) :
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())